# Batch dispersion analysis

## Initialise session and load data

In [ ]:
%load_ext autoreload
%autoreload 2

In [20]:
import warnings                                  
warnings.filterwarnings('ignore')
from IPython.display import display, HTML
import matplotlib.pyplot as plot
import matplotlib.colors
from os.path import join
import pprint
pp = pprint.PrettyPrinter(indent=4)

from src.data.data import *
data = data_wrapper()

deliveries_path = join(data.interimDirectory, 'deliveries', 'deliveries.yaml')
with open(deliveries_path, 'r') as deliveries_yaml:
    print (f'Loading deliveries file from: {deliveries_path}')
    deliveries = yaml.load(deliveries_yaml, Loader=yaml.SafeLoader)

Loading configuration file from: /Users/macoscar/Documents/04_Projects/02_FabLab/01_SmartCitizen/01_Repositories/DataAnalysis/smartcitizen-iscape-data/src/config.yaml
Loaded configuration file
Loading devices data file from: /Users/macoscar/Documents/04_Projects/02_FabLab/01_SmartCitizen/01_Repositories/DataAnalysis/smartcitizen-iscape-data/data/interim
Data initialisation done
Loading deliveries file from: /Users/macoscar/Documents/04_Projects/02_FabLab/01_SmartCitizen/01_Repositories/DataAnalysis/smartcitizen-iscape-data/data/interim/deliveries/deliveries.yaml


In [21]:
# INPUT DATA
# Name of test to be analysed
delivery = 'ISGLOBAL'
batch = 4
type_file = None
# Percentage of points to be considered NG sensor
limit_errors = 3
# Multiplier for std_dev (sigma) - Normal distribution (99.73%)
limit_confidence_sigma = 3
# t-student confidence level (%)
t_confidence_level = 99
# Use average dispersion or instantaneous
use_instantatenous_dispersion = False
# Min/max date for the analysis
# min_date = '2019-12-10 18:00:00'
min_date = None
max_date = None
# In case there is a device with lower amount of channels, ignore the missing channels and keep going
ignore_missing_channels = True
# Smooth channels
smooth_channels = True
smooth_number = 5
## Info of this delivery
pp.pprint (deliveries[delivery]['batches'][batch])

{   'completed': False,
    'date': '2020-01-22',
    'enclosure': 'hdpe',
    'extras': None,
    'file': 'ISGLOBAL/4.csv',
    'power_supply': 'solar',
    'report': None,
    'sensors_delivered': 53,
    'sensors_tested': 55}


In [ ]:
data.load_test(dispersion_test, options = {'clean_na': True, 'clean_na_method': 'drop'})

In [ ]:
print (data.tests['2019-12_INT_ISGLOBAL_22_KITS_RECALL'].devices.keys())


## Get list of common channels
Displays a warning in case there is a device that has fewer channels than the rest. You can choose whether or not to ignore it or update the list of common channels

In [ ]:
# Get list of common channels
save_path = join(data.dataDirectory, 'export/figs', dispersion_test)
# Test Path
if not exists(save_path):
    print ('Creating export directory:\n{}'.format(save_path))
    mkdir(save_path)

list_channels = list()
# Get list of devices
list_devices = list(data.tests[dispersion_test].devices.keys())

In [ ]:
# Init list of common channels. Get the one that has the most
list_channels = data.tests[dispersion_test].devices[list_devices[0]].readings.columns
# Extract list of common channels
len_channels = len(list_channels)

for device in list_devices:
    
    if ignore_missing_channels: 
        # We don't reduce the list in case the new list is smaller
        list_channels = list(set(list_channels) | set(data.tests[dispersion_test].devices[device].readings.columns))
    else:
        # We reduce it
        list_channels = list(set(list_channels) & set(data.tests[dispersion_test].devices[device].readings.columns))

    print ('Device {}'.format(device))
    print ('\tMin reading at {}'.format(data.tests[dispersion_test].devices[device].readings.index[0]))
    #min_date_records = min(min_date_records, records.readings[dispersion_test]['devices'][device]['data'].index[0])
    print ('\tMax reading at {}'.format(data.tests[dispersion_test].devices[device].readings.index[-1]))
    #max_date_records = min(max_date_records, records.readings[dispersion_test]['devices'][device]['data'].index[-1])
    print ('\tNumber of points {}'.format(len(data.tests[dispersion_test].devices[device].readings.index)))
    ## Eliminate devices with no points
    if (len(data.tests[dispersion_test].devices[device].readings.index) == 0):
        print ('Droping device {} for insufficient data points'.format(device))
        data.tests[dispersion_test].devices.pop(device)
    # Check the number of channels    
    elif len_channels != len(data.tests[dispersion_test].devices[device].readings.columns): 
        print("\tWARNING: Device {} has {}. Current common list length is {}".format(device, len(data.tests[dispersion_test].devices[device].readings.columns), len_channels))
        len_channels = len(list_channels)
        if ignore_missing_channels:
            print ("\tIgnoring missing channels")

print('Final list of channels:\n', list_channels)

if min_date is not None: min_date = pd.to_datetime(min_date).tz_localize('UTC').tz_convert('Europe/Madrid')
if max_date is not None: max_date = pd.to_datetime(max_date).tz_localize('UTC').tz_convert('Europe/Madrid')

## Data

In [ ]:
display(HTML('<p><strong>Test name:</strong><br> {}</p>'.format(dispersion_test)))
display(HTML('<p><strong>Total number of devices tested:</strong><br> {}</p>'.format(len(list_devices))))
display(HTML('<p><strong>Test dates:</strong><br> {} - {}</p>'.format(min_date, max_date)))
display(HTML('<p><strong>Test author(s):</strong><br> {}</p><hr>'.format('Óscar González - Victor Barberán')))

### Test Explanation

The devices are co-located for a period of at least 3 days in an **indoor** environment. Devices that show an abnormal behaviour are analysed and replaced if necessary. **In this case, the devices were tested for 2+ weeks during Christmas holidays**.

#### Comment on methods
- This analysis is conditioned to the low amount of devices tested. The plots below indicate only trends and highlight some (but not all) potential faulty devices.
- Below, the confidence intervals used are those of the normal distribution (sample numbers >30) and of the t-student distribution (sample numbers <30).
The testing of the devices is done for at least 3-5 days. If any problem is seen, longer periods are tested if needed. Also note that statistical significance cannot be inferred from these amounts of devices, and some assumptions have to be accepted. Comments and further discussion are always welcome.
- The individual sensors components integrated in the Smart Citizen hardware have their own accuracies and dispersions, for which Smart Citizen cannot assume any liability other than trying to work with the most appropiate selection. The tests we perform are aimed to determine and assume any failures in the sensors and their integration within the Smart Citizen hardware. For more information, please check the <a href="https://docs.smartcitizen.me">official documentation</a> and the datasheets of each of the sensors in the [sensors part](https://docs.smartcitizen.me/Components/Urban%20Sensor%20Board/).

In [ ]:
from scipy import stats
from scipy.stats import t
import numpy as np
import traceback

# Calculate the dispersion for the sensors present in the dataset
dispersion_df = pd.DataFrame()
dispersion_history = list()
display(HTML('<h3>Warnings</h3>'))
warning_displayed = False
location = None

for device in list_devices:
    location_test = data.tests[dispersion_test].devices[device].location
    if location_test is None: data.std_out (f'Device {device} has no location')
    else:
        if location is None: location = location_test
        elif location_test != location: data.std_out (f'Device {device} has different location!')
        
for channel in list_channels:
    list_columns = list()
    if channel != 'BATT':
        for device in list_devices:
            if channel in data.tests[dispersion_test].devices[device].readings.columns and len(data.tests[dispersion_test].devices[device].readings.loc[:,channel]) >0 :
                # Important to resample and bfill for unmatching measures
                if smooth_channels:
                    channel_new = data.tests[dispersion_test].devices[device].readings[channel].resample('1Min').bfill().rolling(window=smooth_number).mean()
                    dispersion_df[channel + '-' + device] = channel_new[channel_new > 0]
                else:
                    dispersion_df[channel + '-' + device] = data.tests[dispersion_test].devices[device].readings[channel].resample('1Min').bfill()
                
                list_columns.append(channel + '-' + device)
            else:
                display(HTML('<p>WARNING: Device {} does not contain {}</p>'.format(device, channel)))
                warning_displayed = True
    try:
        if dispersion_df.index.tzinfo is None: dispersion_df.index = dispersion_df.index.tz_localize('UTC').tz_convert(location)
    except:
        traceback.print_exc()
        pass
    
    # Trim dataset to min and max dates (normally these tests are carried out with _minutes_ of differences)
    if min_date is not None: dispersion_df = dispersion_df[dispersion_df.index > min_date]
    if max_date is not None: dispersion_df = dispersion_df[dispersion_df.index < max_date]

    # Calculate Metrics
    dispersion_df[channel + '_AVG'] = dispersion_df.loc[:,list_columns].mean(skipna=True, axis = 1)
    dispersion_df[channel + '_STD'] = dispersion_df.loc[:,list_columns].std(skipna=True, axis = 1)
 
    # Calculate Metrics
    dispersion_global = dispersion_df[channel + '_STD'].mean()
    # print (dispersion_df.index[0], dispersion_df.index[-1], channel, dispersion_global)
    dispersion_history.append([channel, dispersion_global])
if not warning_displayed:
    display(HTML('<p>All devices show similar amounts of data. No data loss concern</p>'.format(device, channel)))
    
# display(HTML('<h3>Sensor dispersion</h3>'))
# display(HTML('<p>Below, the sensor dispersion for each channel is listed (units of each sensor)</p>'))
# dispersion_history = tuple(dispersion_history)
# display(HTML('<ul style="list-style-type:disc;">'))
# for item in dispersion_history:
#     display(HTML('<li>{}: {}</li>'.format(item[0], item[1])))
# display(HTML('</ul>'))


In [ ]:
display(HTML('<hr>'))
display(HTML('<div class="pagebreak"> </div>'))

### Internal notes:
    
+ iSCAPE:
    - 10408: pm sensor ??
    - 10413: pm sensor A
    - 10420: pm sensor A 
    - 10409: pm sensor B
     
    - Sensor resetting issue:
        1. 10422 1 PM Disconnected (11-11 19h)
        2. 10421 Gases Pro Board Disconnected (11-11 19h)
        3. Added battery to 9994 (12-11 13h)

## Conclusions



- 4E77: Wasn't publishing - 10621: no battery (not connected to power supply) --> 👍🏽
- 2EC0: Hasn't published PM - 10619: PM sensor was disconnected --> 👍🏽
- FD94: Only published battery -  10615. Shows signs of water corrosion - Replaced full kit by a new one (not PM sensor) --> 👍🏽

<div style="text-align: left" >
<img src="https://i.imgur.com/crIy7Cz.jpg" width=500px>
</div>

- 7FCB: Wasn't publishing - 10607: no battery (got disconnected by accident from power supply). The CCS811 was damaged (see below). After replacement of urban board. Full period shows good behaviour. --> 👍🏽

<div style="text-align: left" >
<img src="https://i.imgur.com/s3IeuyB.jpg" width=500px>
</div>

- All eCO2/tVOC sensors show high dispersion values in _two or more_ distinct behaviours. After 8/01/2020 they start converging, but they still show high dispersion. No action is taken, unless is requested to. This shows that, probably, if the sensors were set in different environments, their behaviour with respect to the baseline takes longer than the manufacturer says to stabilise.
- All PM sensors show good behaviour
- Three sensors show warnings in humidity: two of them are due to no publication. One of them shows higher dispersion with ~5%rh. No action is taken as the value somewhat near the +-2%rh typical accuracy of the sensor, accounting for the possible test bias

In [ ]:
display(HTML('<hr>'))

## Time Series Plot (Full Period)

In [ ]:
list_channels_kit =  ['PRESS', 'CCS811_ECO2', 'EXT_PM_10', 'NOISE_A', 'TEMP', 'CCS811_VOCS', 'HUM', 'EXT_PM_1', 'LIGHT', 'EXT_PM_25']
test_for_kit = False
list_channels_plots = list_channels_kit if test_for_kit else list_channels

In [ ]:
display(HTML('<h2>Time Series Plots</h2>'))
min_date = '2019-12-15'
if min_date is None: min_date_test = dispersion_df.index[-1] 
else: min_date_test = min_date
if max_date is None: max_date_test = dispersion_df.index[-1] 
else: max_date_test = max_date
display(HTML('Min Date available: {}'.format(min_date_test)))
display(HTML('Max Date available: {}'.format(max_date_test)))

dispersion_df_trim = dispersion_df.copy()
dispersion_df_trim = dispersion_df_trim[dispersion_df_trim.index > min_date_test]
dispersion_df_trim = dispersion_df_trim[dispersion_df_trim.index < max_date_test]

# Ignore battery
if 'BATT' in list_channels_plots: list_channels_plots.remove('BATT')
dict_devices_tbr = dict()
for item in list_channels_plots: dict_devices_tbr[item] = list()

for channel in list_channels_plots:
    if channel not in list_channels_plots and test_for_kit: continue
    # Make subplot
    list_columns = list()
    fig, (ax1, ax2) = plot.subplots(nrows = 2, figsize= (15,10))
    cmap = plot.cm.Reds
    norm = matplotlib.colors.Normalize(vmin=0, vmax=limit_errors/2)
    index = list_channels_plots.index(channel)+1
    total_number = len(list_channels_plots)
    display(HTML('<h3>({}/{}) - {} </h3>'.format(index, total_number, channel)))
    
    dispersion_avg = 0
    limit_confidence_sigma = 0
    for item in dispersion_history:
        if channel == item[0]:
            dispersion_avg = item[1]
            
    if len(list_devices)>30:
        display(HTML('<p>Using Normal Distribution. Using limit for sigma confidence: {}'.format(limit_confidence_sigma)))
        limit_confidence = limit_confidence_sigma
        # Calculate upper and lower bounds
        if (use_instantatenous_dispersion):
            # For sensors with high variability in the measurements, it's better to use this (i.e. alphasense)
            upper_bound = dispersion_df_trim[channel + '_AVG'] + limit_confidence * dispersion_df_trim[channel + '_STD']
            lower_bound = dispersion_df_trim[channel + '_AVG'] - abs(limit_confidence * dispersion_df_trim[channel + '_STD'])
        else:
            upper_bound = dispersion_df_trim[channel + '_AVG'] + limit_confidence * dispersion_avg
            lower_bound = dispersion_df_trim[channel + '_AVG'] - abs(limit_confidence * dispersion_avg)
    else:
        limit_confidence = t.interval(t_confidence_level/100.0, len(list_devices), loc=dispersion_df_trim[channel + '_AVG'], scale=dispersion_avg)
        display(HTML('<p>Using t-student Distribution</p>'))
        upper_bound = limit_confidence[1]
        lower_bound = limit_confidence[0]

    dispersion_df_trim[channel + '_MAX'] = dispersion_df_trim.loc[:,list_columns].max(skipna=True, axis = 1)
    dispersion_df_trim[channel + '_MIN'] = dispersion_df_trim.loc[:,list_columns].min(skipna=True, axis = 1)
        
    # print ('Plotting devices')
    for device in list_devices:
        name_column = channel + '-' + device 
        if name_column in dispersion_df_trim.columns:
            # Count how many times we go above the upper bound or below the lower one
            count_problems_up = dispersion_df_trim[name_column] > upper_bound
            count_problems_down =  dispersion_df_trim[name_column] < lower_bound
            
            # Count them
            count_problems = [1 if (count_problems_up[i] or count_problems_down[i]) else 0 for i in range(len(count_problems_up))]
            # print (channel, device, np.sum(count_problems), len(count_problems))
            
            # Add the trace in either
            number_errors = np.sum(count_problems)
            max_number_errors = len(count_problems)
            
            if number_errors/max_number_errors > limit_errors/100:
                print (f'WARNING: Device {device} out of {limit_errors}% limit - {np.round(number_errors/max_number_errors*100, 1)}% out')
                if device not in dict_devices_tbr[channel]: dict_devices_tbr[channel].append(device)
                alpha = 1
                ax1.plot(dispersion_df_trim.index, 
                         dispersion_df_trim[name_column], 
                         color = 'r',
                         label = device, alpha = alpha)
            else:
                alpha = 1
                color = 'g'
                ax2.plot(dispersion_df_trim.index, 
                         dispersion_df_trim[name_column], 
                         color = color, 
                         label = device, alpha = alpha)
        
    # Add upper and low bound bound to subplot 1
    ax1.plot(dispersion_df_trim.index, dispersion_df_trim[channel + '_AVG'],'b', label = 'Average', alpha = 0.6)
    ax1.plot(dispersion_df_trim.index, upper_bound, 'k', label = 'Upper-Bound', alpha = 0.6)
    ax1.plot(dispersion_df_trim.index, lower_bound, 'k',label = 'Lower-Bound', alpha = 0.6)
    
    # Format the legend
    lgd1 = ax1.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    ax1.grid(True)
    ax1.set_ylabel(channel + ' TBR')
    ax1.set_xlabel('Time')
    
    # Add upper and low bound bound to subplot 2
    ax2.plot(dispersion_df_trim.index, dispersion_df_trim[channel + '_AVG'],'b', label = 'Average', alpha = 0.6)
    ax2.plot(dispersion_df_trim.index, upper_bound, 'k', label = 'Upper-Bound', alpha = 0.6)
    ax2.plot(dispersion_df_trim.index, lower_bound, 'k',label = 'Lower-Bound', alpha = 0.6)
    
    # Format the legend
    ax2.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    lgd2 = ax2.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    ax2.grid(True)
    ax2.set_ylabel(channel + ' OK')
    ax2.set_xlabel('Time')
    
    # Check file type to make the export
    if type_file is not None: print ('Saving figure')
    if type_file == 'fig':
        pickle.dump(fig, open(save_path + '/' + dispersion_test + '_' + channel + '.fig.pickle', 'wb'))
    elif type_file == 'png':
        fig.savefig(save_path + '/' + dispersion_test + '_' + channel + '.png', dpi=300, trasnparent = True, bbox_extra_artists=(lgd1, lgd2), bbox_inches='tight' )

    # Show plots     
    plot.show()
    display(HTML('<br>'))

### Summary

In [ ]:
summary_df = pd.DataFrame(index = list_channels_plots)

for item in dispersion_history:
    summary_df.loc[item[0], 'Dispersion'] = item[1]
    if item[0] != 'BATT':
        summary_df.loc[item[0], 'Total Number of devices'] = len(list_devices)
        summary_df.loc[item[0], 'TBR Number of devices'] = len(dict_devices_tbr[item[0]])
        summary_df.loc[item[0], 'OK Number of devices'] = len(list_devices) - len(dict_devices_tbr[item[0]])
    
display (summary_df)

## Time Series Plot (After fixes)

In [ ]:
display(HTML('<h2>Time Series Plots</h2>'))
min_date = '2020-01-08'
if min_date is None: min_date_test = dispersion_df.index[-1] 
else: min_date_test = min_date
if max_date is None: max_date_test = dispersion_df.index[-1] 
else: max_date_test = max_date
display(HTML('Min Date available: {}'.format(min_date_test)))
display(HTML('Max Date available: {}'.format(max_date_test)))

dispersion_df_trim = dispersion_df.copy()
dispersion_df_trim = dispersion_df_trim[dispersion_df_trim.index > min_date_test]
dispersion_df_trim = dispersion_df_trim[dispersion_df_trim.index < max_date_test]

# Ignore battery
if 'BATT' in list_channels_plots: list_channels_plots.remove('BATT')
dict_devices_tbr = dict()
for item in list_channels_plots: dict_devices_tbr[item] = list()

for channel in list_channels_plots:
    if channel not in list_channels_plots and test_for_kit: continue
    # Make subplot
    list_columns = list()
    fig, (ax1, ax2) = plot.subplots(nrows = 2, figsize= (15,10))
    cmap = plot.cm.Reds
    norm = matplotlib.colors.Normalize(vmin=0, vmax=limit_errors/2)
    index = list_channels_plots.index(channel)+1
    total_number = len(list_channels_plots)
    display(HTML('<h3>({}/{}) - {} </h3>'.format(index, total_number, channel)))
    
    dispersion_avg = 0
    limit_confidence_sigma = 0
    for item in dispersion_history:
        if channel == item[0]:
            dispersion_avg = item[1]
            
    if len(list_devices)>30:
        display(HTML('<p>Using Normal Distribution. Using limit for sigma confidence: {}'.format(limit_confidence_sigma)))
        limit_confidence = limit_confidence_sigma
        # Calculate upper and lower bounds
        if (use_instantatenous_dispersion):
            # For sensors with high variability in the measurements, it's better to use this (i.e. alphasense)
            upper_bound = dispersion_df_trim[channel + '_AVG'] + limit_confidence * dispersion_df_trim[channel + '_STD']
            lower_bound = dispersion_df_trim[channel + '_AVG'] - abs(limit_confidence * dispersion_df_trim[channel + '_STD'])
        else:
            upper_bound = dispersion_df_trim[channel + '_AVG'] + limit_confidence * dispersion_avg
            lower_bound = dispersion_df_trim[channel + '_AVG'] - abs(limit_confidence * dispersion_avg)
    else:
        limit_confidence = t.interval(t_confidence_level/100.0, len(list_devices), loc=dispersion_df_trim[channel + '_AVG'], scale=dispersion_avg)
        display(HTML('<p>Using t-student Distribution</p>'))
        upper_bound = limit_confidence[1]
        lower_bound = limit_confidence[0]

    dispersion_df_trim[channel + '_MAX'] = dispersion_df_trim.loc[:,list_columns].max(skipna=True, axis = 1)
    dispersion_df_trim[channel + '_MIN'] = dispersion_df_trim.loc[:,list_columns].min(skipna=True, axis = 1)
        
    # print ('Plotting devices')
    for device in list_devices:
        name_column = channel + '-' + device 
        if name_column in dispersion_df_trim.columns:
            # Count how many times we go above the upper bound or below the lower one
            count_problems_up = dispersion_df_trim[name_column] > upper_bound
            count_problems_down =  dispersion_df_trim[name_column] < lower_bound
            
            # Count them
            count_problems = [1 if (count_problems_up[i] or count_problems_down[i]) else 0 for i in range(len(count_problems_up))]
            # print (channel, device, np.sum(count_problems), len(count_problems))
            
            # Add the trace in either
            number_errors = np.sum(count_problems)
            max_number_errors = len(count_problems)
            
            if number_errors/max_number_errors > limit_errors/100:
                print (f'WARNING: Device {device} out of {limit_errors}% limit - {np.round(number_errors/max_number_errors*100, 1)}% out')
                if device not in dict_devices_tbr[channel]: dict_devices_tbr[channel].append(device)
                alpha = 1
                ax1.plot(dispersion_df_trim.index, 
                         dispersion_df_trim[name_column], 
                         color = 'r',
                         label = device, alpha = alpha)
            else:
                alpha = 1
                color = 'g'
                ax2.plot(dispersion_df_trim.index, 
                         dispersion_df_trim[name_column], 
                         color = color, 
                         label = device, alpha = alpha)
        
    # Add upper and low bound bound to subplot 1
    ax1.plot(dispersion_df_trim.index, dispersion_df_trim[channel + '_AVG'],'b', label = 'Average', alpha = 0.6)
    ax1.plot(dispersion_df_trim.index, upper_bound, 'k', label = 'Upper-Bound', alpha = 0.6)
    ax1.plot(dispersion_df_trim.index, lower_bound, 'k',label = 'Lower-Bound', alpha = 0.6)
    
    # Format the legend
    lgd1 = ax1.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    ax1.grid(True)
    ax1.set_ylabel(channel + ' TBR')
    ax1.set_xlabel('Time')
    
    # Add upper and low bound bound to subplot 2
    ax2.plot(dispersion_df_trim.index, dispersion_df_trim[channel + '_AVG'],'b', label = 'Average', alpha = 0.6)
    ax2.plot(dispersion_df_trim.index, upper_bound, 'k', label = 'Upper-Bound', alpha = 0.6)
    ax2.plot(dispersion_df_trim.index, lower_bound, 'k',label = 'Lower-Bound', alpha = 0.6)
    
    # Format the legend
    ax2.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    lgd2 = ax2.legend(bbox_to_anchor=(1.1, 0.5), fancybox=True, loc='center left', ncol = 5)
    ax2.grid(True)
    ax2.set_ylabel(channel + ' OK')
    ax2.set_xlabel('Time')
    
    # Check file type to make the export
    if type_file is not None: print ('Saving figure')
    if type_file == 'fig':
        pickle.dump(fig, open(save_path + '/' + dispersion_test + '_' + channel + '.fig.pickle', 'wb'))
    elif type_file == 'png':
        fig.savefig(save_path + '/' + dispersion_test + '_' + channel + '.png', dpi=300, trasnparent = True, bbox_extra_artists=(lgd1, lgd2), bbox_inches='tight' )

    # Show plots     
    plot.show()
    display(HTML('<br>'))